In [ ]:
import pandas as pd
import numpy as np
import cbsodata
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import os


# Sample size
n = 10000

# Random seed for reproducibility
random_seed = 1234567

# Target variable parameters
target_intercept = -5
target_noise_sd = 1.25
target_coef_scale = 0.15

# Feature coefficients for target variable Y
target_coefficients = {
    'x1': 0.8,   # Strong positive effect
    'x2': 0.6,   # Moderate positive effect  
    'x3': -0.4,  # Moderate negative effect
    'x4': 0.8,   # Strong positive effect
    'x5': 0.05,  # Weak positive effect
    'x6': 0.3,   # Moderate positive effect
    'x7': 0.2,   # Weak positive effect
    'x8': 0.1,   # Weak positive effect
    'x9': 0.03,  # Very weak positive effect
    'x10': 0.4   # Moderate positive effect
}

# Feature generation parameters
feature_params = {
    # Binary features
    'x1': {'base_prob': 0.3, 'gender_effect': 0.2, 'nationality_effect': 0.1},
    'x2': {'base_prob': 0.4, 'age_effect': 0.15, 'age_groups': ["25 tot 30 jaar", "30 tot 35 jaar", "35 tot 40 jaar"]},
    'x3': {'base_prob': 0.5, 'nationality_effect': -0.2},
    'x4': {'base_prob': 0.35, 'gender_effect': 0.1, 'age_effect': -0.05, 'age_groups': ["65 tot 70 jaar", "70 tot 75 jaar"]},
    
    # Continuous features
    'x5': {'base_mean': 10, 'gender_effect': 2, 'nationality_effect': -1.5, 'sd': 3},
    'x6': {'base_mean': 50, 'age_effect': 10, 'age_groups': ["45 tot 50 jaar", "50 tot 55 jaar", "55 tot 60 jaar", "60 tot 65 jaar", "65 tot 70 jaar", "70 tot 75 jaar"], 'sd': 15},
    'x7': {'base_mean': 20, 'nationality_effect': -5, 'gender_effect': 2, 'sd': 8},
    'x8': {'base_mean': 5, 'age_effect': 3, 'age_groups': ["15 tot 20 jaar", "20 tot 25 jaar"], 'sd': 2},
    
    # Count features  
    'x9': {'base_lambda': 2, 'gender_effect': 1.5, 'nationality_effect': 0.8},
    'x10': {'base_lambda': 1.5, 'age_effect': 0.5, 'age_groups': ["30 tot 35 jaar", "35 tot 40 jaar", "40 tot 45 jaar"]}
}

# Set seed for reproducibility
np.random.seed(random_seed)

In [58]:
# Fetch and prepare CBS demographic data
print("Fetching CBS data...")
cbs_data = pd.DataFrame(cbsodata.get_data('03743'))

In [ ]:
# Filter and prepare data
data = cbs_data.loc[
    (cbs_data['Perioden'] == "2023") &  # only look at year 2023
    (cbs_data['Nationaliteiten'].isin(["Nederlands", "Totaal niet-Nederlandse nationaliteit"])) &
    (cbs_data['Leeftijd'].isin([
        "15 tot 20 jaar", "20 tot 25 jaar", "25 tot 30 jaar", "30 tot 35 jaar",
        "35 tot 40 jaar", "40 tot 45 jaar", "45 tot 50 jaar", "50 tot 55 jaar",
        "55 tot 60 jaar", "60 tot 65 jaar", "65 tot 70 jaar", "70 tot 75 jaar"
    ])) &
    (cbs_data['Geslacht'].isin(["Mannen", "Vrouwen"]))
].copy()

In [ ]:
# Select relevant columns and calculate probabilities
data = data[['Nationaliteiten', 'Geslacht', 'Leeftijd', 'BevolkingOp1Januari_1']].copy()
data['prob'] = data['BevolkingOp1Januari_1'] / data['BevolkingOp1Januari_1'].sum()
data['combination'] = data['Nationaliteiten'] + ";" + data['Geslacht'] + ";" + data['Leeftijd']

In [ ]:
print("Generating simulated demographic data...")

# Generate simulated demographic data from aggregated CBS data
combinations = np.random.choice(data['combination'].values, size=n, replace=True, p=data['prob'].values)

# Split combinations back into separate columns
simulated_data = pd.DataFrame({
    'combination': combinations
})

split_data = simulated_data['combination'].str.split(';', expand=True)
split_data.columns = ['Nationaliteiten', 'Geslacht', 'Leeftijd']
simulated_data = pd.concat([simulated_data, split_data], axis=1)

# Create dummy variables
dummy_data = pd.get_dummies(simulated_data, columns=['Nationaliteiten', 'Geslacht', 'Leeftijd'])
simulated_data = pd.concat([simulated_data, dummy_data], axis=1)

In [ ]:
print("Generating features X1-X10...")

# Generate X1-X10 features correlated with demographics using parameters
# Binary features
simulated_data['x1'] = np.random.binomial(1, np.clip(
    feature_params['x1']['base_prob'] + 
    feature_params['x1']['gender_effect'] * simulated_data['Geslacht_Mannen'] + 
    feature_params['x1']['nationality_effect'] * simulated_data['Nationaliteiten_Nederlands'],
    0, 1), n)

age_x2_mask = simulated_data[[col for col in simulated_data.columns if any(age in col for age in feature_params['x2']['age_groups'])]].sum(axis=1) > 0
simulated_data['x2'] = np.random.binomial(1, np.clip(
    feature_params['x2']['base_prob'] + 
    feature_params['x2']['age_effect'] * age_x2_mask.astype(int), 
    0, 1), n)

simulated_data['x3'] = np.random.binomial(1, np.clip(
    feature_params['x3']['base_prob'] + 
    feature_params['x3']['nationality_effect'] * simulated_data['Nationaliteiten_Nederlands'],
    0, 1), n)

age_x4_mask = simulated_data[[col for col in simulated_data.columns if any(age in col for age in feature_params['x4']['age_groups'])]].sum(axis=1) > 0
simulated_data['x4'] = np.random.binomial(1, np.clip(
    feature_params['x4']['base_prob'] + 
    feature_params['x4']['gender_effect'] * simulated_data['Geslacht_Mannen'] + 
    feature_params['x4']['age_effect'] * age_x4_mask.astype(int),
    0, 1), n)

# Continuous features
simulated_data['x5'] = np.random.normal(
    feature_params['x5']['base_mean'] + 
    feature_params['x5']['gender_effect'] * simulated_data['Geslacht_Mannen'] + 
    feature_params['x5']['nationality_effect'] * simulated_data['Nationaliteiten_Nederlands'], 
    feature_params['x5']['sd'], n)

age_x6_mask = simulated_data[[col for col in simulated_data.columns if any(age in col for age in feature_params['x6']['age_groups'])]].sum(axis=1) > 0
simulated_data['x6'] = np.random.normal(
    feature_params['x6']['base_mean'] + 
    feature_params['x6']['age_effect'] * age_x6_mask.astype(int), 
    feature_params['x6']['sd'], n)

simulated_data['x7'] = np.random.normal(
    feature_params['x7']['base_mean'] + 
    feature_params['x7']['nationality_effect'] * simulated_data['Nationaliteiten_Nederlands'] + 
    feature_params['x7']['gender_effect'] * simulated_data['Geslacht_Mannen'], 
    feature_params['x7']['sd'], n)

age_x8_mask = simulated_data[[col for col in simulated_data.columns if any(age in col for age in feature_params['x8']['age_groups'])]].sum(axis=1) > 0
simulated_data['x8'] = np.random.normal(
    feature_params['x8']['base_mean'] + 
    feature_params['x8']['age_effect'] * age_x8_mask.astype(int), 
    feature_params['x8']['sd'], n)

# Count features
simulated_data['x9'] = np.random.poisson(np.clip(
    feature_params['x9']['base_lambda'] + 
    feature_params['x9']['gender_effect'] * simulated_data['Geslacht_Mannen'] + 
    feature_params['x9']['nationality_effect'] * simulated_data['Nationaliteiten_Nederlands'],
    0, None), n)

age_x10_mask = simulated_data[[col for col in simulated_data.columns if any(age in col for age in feature_params['x10']['age_groups'])]].sum(axis=1) > 0
simulated_data['x10'] = np.random.poisson(np.clip(
    feature_params['x10']['base_lambda'] + 
    feature_params['x10']['age_effect'] * age_x10_mask.astype(int),
    0, None), n)

In [ ]:
print("Generating target variable Y...")

# Target variable Y with noise using parameters
noise = np.random.normal(0, target_noise_sd, n)
linear_combination = target_intercept + target_coef_scale * (
    target_coefficients['x1'] * simulated_data['x1'] + 
    target_coefficients['x2'] * simulated_data['x2'] + 
    target_coefficients['x3'] * simulated_data['x3'] + 
    target_coefficients['x4'] * simulated_data['x4'] + 
    target_coefficients['x5'] * simulated_data['x5'] + 
    target_coefficients['x6'] * simulated_data['x6'] + 
    target_coefficients['x7'] * simulated_data['x7'] + 
    target_coefficients['x8'] * simulated_data['x8'] + 
    target_coefficients['x9'] * simulated_data['x9'] + 
    target_coefficients['x10'] * simulated_data['x10']
) + noise

# Convert to probabilities using logistic function and generate binary outcomes
probabilities = 1 / (1 + np.exp(-linear_combination))
simulated_data['y'] = np.random.binomial(1, probabilities, n)


In [ ]:
print("Fitting logistic regression model...")

# Fit logistic regression model to predict Y using X1-X10
feature_columns = [f'x{i}' for i in range(1, 11)]
X = simulated_data[feature_columns]
y = simulated_data['y']

model = LogisticRegression(random_state=random_seed, max_iter=1000)
model.fit(X, y)

# Print model coefficients (similar to R's summary)
print("\nLogistic Regression Results:")
print("=" * 50)
print("Coefficients:")
for i, coef in enumerate(model.coef_[0], 1):
    print(f"x{i}: {coef:.6f}")
print(f"Intercept: {model.intercept_[0]:.6f}")

# Model performance summary
print(f"\nPositive cases: {sum(simulated_data['y'])} out of {len(simulated_data)}")
print(f"Proportion of positive cases: {np.mean(simulated_data['y']):.4f}")


In [ ]:
# Ensure data directory exists
os.makedirs("../data", exist_ok=True)

# Export dataset to CSV (exclude intermediate variables)
export_columns = ['Nationaliteiten', 'Geslacht', 'Leeftijd'] + feature_columns + ['y']
export_data = simulated_data[export_columns]

export_data.to_csv("../data/simulated_data.csv", index=False)
print("Dataset exported to ../data/simulated_data.csv")